
# <font color=blue><div align="center">ST7 SNCF : Jalon 1</div></font>

## <font color=blue><div align="center">"<em>Indian Railways</em>"</div></font>

## Modules

In [432]:
# Modules de base
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import datetime
%matplotlib inline

# Module relatif à Gurobi
from gurobipy import *

# Module csv
import csv

La modélisation mathématique est plus explicitée en détail dans le rapport

In [433]:
mini_instance="mini_instance.xlsx"
instance_simple="instance_WPY_simple.xlsx"
instance_realiste="instance_WPY_realiste_jalon2.xlsx"

# CHOIX DE L'INTSANCE 
instance=mini_instance

## Traitement de données

In [434]:
def excel_to_dict(file: str) -> dict:
    '''
    Takes excel file name in the repo
    and returns a dictionary[sheet_name]=Dataframe
    '''
    data_dict=pd.read_excel(file, sheet_name=None)
    return data_dict


def date_to_creneau(date0: datetime.datetime, hour: pd.DataFrame, date: pd.DataFrame):
    '''
    Takes hour and date's 1-column dataframes
    and returns a dataframe of 1-column of 'créneaux'
    starting from date0
    '''
    # Excel is sometimes stoopid and has inconsistent types
    if type(date[0])==str:
        datetime_df=pd.to_datetime(date.apply(lambda x: str(x)+' ')+hour.apply(lambda x:str(x)), dayfirst=True)
    if type(date[0])==pd._libs.tslibs.timestamps.Timestamp:
        datetime_df=pd.to_datetime(date.apply(lambda x: str(x)+' ')+hour.apply(lambda x:str(x)))
    return (datetime_df-date0).dt.total_seconds().div(60*15).astype(int)+1

def creneau_to_date(date0: datetime.datetime, creneau : int):
    return date0+ datetime.timedelta(minutes=(creneau-1)*15)

def convToStr(date)-> str:
    '''
    Takes a date in any form in the instance
    and returns a string dd/mm/yyyy
    '''
    if type(date)==str:
        return date
    else:
        return "/".join(str(date.date()).split("-")[::-1])
    

INSTANCE= excel_to_dict(instance)

### Dictionnaires utiles 

Trains de l'arrivée et trains du départ

In [435]:
trains_arrivee=[]
for index, row in INSTANCE["Sillons arrivee"].iterrows():
    id_train = row['n°TRAIN'], convToStr(row["JARR"])
    trains_arrivee.append(id_train)

trains_depart=[]
for index, row in INSTANCE["Sillons depart"].iterrows():
    id_train = row['n°TRAIN'], convToStr(row["JDEP"])
    trains_depart.append(id_train)

Créneaux de l'arrivée et du départ

In [436]:
# Date de référence est celle du lundi de la première semaine
# Ce choix rend facile la traduction des indisponibilités de type (weekday,hh:mm)
date0=str(INSTANCE["Sillons arrivee"]["JARR"][0])+' 00:00:00'
date0=pd.to_datetime(date0, dayfirst=True)
date0=date0-datetime.timedelta(days=float(date0.dayofweek))
datef=pd.to_datetime(INSTANCE["Sillons depart"]["JDEP"].iloc[-1], dayfirst=True)+datetime.timedelta(hours=24)
nombre_creneaux=int((datef-date0).total_seconds()/(60*15))

# Créneaux du départ et de l'arrivée
INSTANCE["Sillons arrivee"]["creneau"]=date_to_creneau(date0, INSTANCE["Sillons arrivee"]["HARR"], INSTANCE["Sillons arrivee"]["JARR"])
INSTANCE["Sillons depart"]["creneau"]=date_to_creneau(date0, INSTANCE["Sillons depart"]["HDEP"], INSTANCE["Sillons depart"]["JDEP"])

# Dictionnaires des créneaux 
# { (n°train, jour) : créneau }
creneau_arrivee={} 
for index, row in INSTANCE["Sillons arrivee"].iterrows():
    creneau_arrivee[row["n°TRAIN"],convToStr(row["JARR"])]=row["creneau"]

creneau_depart={}
for index, row in INSTANCE["Sillons depart"].iterrows():
    creneau_depart[row["n°TRAIN"],convToStr(row["JDEP"])]=row["creneau"]

C:\Users\htmou\AppData\Local\Temp\ipykernel_23072\1452588239.py:20: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  datetime_df=pd.to_datetime(date.apply(lambda x: str(x)+' ')+hour.apply(lambda x:str(x)))


Correspondances

In [437]:
def correspondances():
    '''
    Construit un dictionnaire global
    associant chaque train du départ à un set 
    des trains d'arrivée qui le constituent
    et à un train d'arrivée un set de trains 
    de départs qui lui y sont liés
    '''
    At={}
    Dt={}
    for train in trains_depart:
        At[train]=set()
    for train in trains_arrivee:
         Dt[train]=set()
    for index, row in INSTANCE["Correspondances"].iterrows():
            At[(row["n°Train depart"], convToStr(row["Jour depart"]))].add((row['n°Train arrivee'],convToStr(row['Jour arrivee'])))
            Dt[(row["n°Train arrivee"], convToStr(row["Jour arrivee"]))].add((row['n°Train depart'],convToStr(row['Jour depart'])))
            
    return At,Dt

At=correspondances()[0]
Dt=correspondances()[1]


Indisponibilité des machines et des chantiers

In [438]:
def creneaux_de_periode(indispo_str : str)-> list[int]: 
    plage_jours=nombre_creneaux//96
    creneaux = []
    if indispo_str!='0':
        periodes = indispo_str.split(';')
        for periode in periodes:
            jour, plage_horaire = periode.strip('()').split(',')
            jour = int(jour)
            jours=[jour+7*k for k in range(0,plage_jours//7+1)]
            plage_horaire = plage_horaire.strip('()').split('-')
            heure_debut, minute_debut = map(int, plage_horaire[0].split(':'))
            creneau_debut=minute_debut//15+1
            heure_fin, minute_fin = map(int, plage_horaire[1].split(':'))
            creneau_fin=minute_fin//15+1


            if (heure_debut,minute_debut)>=(heure_fin,minute_fin):
                for jour1 in jours:
                    for c in range((jour1-1)*96+heure_debut*4+creneau_debut,jour1*96):
                        if c<=nombre_creneaux:
                            creneaux.append(c)
                    for c in range(jour1*96,jour1*96+heure_fin*4+creneau_fin):
                        if c<=nombre_creneaux: creneaux.append(c)
            else:
                for jour1 in jours:
                    for c in range((jour1-1)*96+heure_debut*4+creneau_debut,(jour1-1)*96+heure_fin*4+creneau_fin):
                        if c<=nombre_creneaux: creneaux.append(c) 
    return creneaux

# data1 = INSTANCE1['Machines']
df = INSTANCE['Machines']

df['Indisponibilites']=df['Indisponibilites'].astype(str)
# Apply the function to each row
df['Indisponibilites_TimeSlots'] = df['Indisponibilites'].apply(creneaux_de_periode)

DEB_INDIS = df.loc[df['Machine'] == 'DEB', 'Indisponibilites_TimeSlots'].values[0]
FOR_INDIS = df.loc[df['Machine'] == 'FOR', 'Indisponibilites_TimeSlots'].values[0]
DEG_INDIS = df.loc[df['Machine'] == 'DEG', 'Indisponibilites_TimeSlots'].values[0]

Voies disponibles des chantiers

In [439]:
df=INSTANCE["Chantiers"]
df['Indisponibilites']=df['Indisponibilites'].astype(str)
df['Indisponibilites_TimeSlots'] = df['Indisponibilites'].apply(creneaux_de_periode)

REC_INDIS = df.loc[df['Chantier'] == 'WPY_REC', 'Indisponibilites_TimeSlots'].values[0]
OFORM_INDIS = df.loc[df['Chantier'] == 'WPY_FOR', 'Indisponibilites_TimeSlots'].values[0]
DEP_INDIS = df.loc[df['Chantier'] == 'WPY_DEP', 'Indisponibilites_TimeSlots'].values[0]

NVREC=df.loc[df['Chantier'] == 'WPY_REC', "Nombre de voies"].values[0]
NVFOR=df.loc[df['Chantier'] == 'WPY_FOR', "Nombre de voies"].values[0]
NVDEP=df.loc[df['Chantier'] == 'WPY_DEP', "Nombre de voies"].values[0]

Roulements agents

In [440]:
Roulements_source = INSTANCE['Roulements agents']
Roulements = dict()
a = 0
i = 0
for name in Roulements_source['Roulement']:
    Roulements[name] = dict()
    Roulements[name]['Jours'] = Roulements_source['Jours de la semaine'][i].split(';')
    Roulements[name]['Nb agents'] = Roulements_source['Nombre agents'][i]
    Roulements[name]['Blocs'] = Roulements_source['Cycles horaires'][i].split(';')
    Roulements[name]['Connaissance'] = Roulements_source['Connaissances chantiers'][i].split(';')
    Blocs = dict()
    for j in Roulements[name]['Jours']: 
        for b in Roulements[name]['Blocs']:
            Blocs[(j,b)] = list()
            lbrgag = 0 #indicateur de nb de creneaux pour split des blocs de 8h
            for c in creneaux_de_periode(','.join([j, b])):
                lbrgag += 1
                if lbrgag == 33: #bloc de 8h
                    a = str(int(j) + 7) #semaine pro
                    Blocs[(a,b)] = list()
                if lbrgag >= 33: 
                    Blocs[(a,b)].append(c)
                else :
                    Blocs[(j,b)].append(c)
    Roulements[name]['Creneaux'] = Blocs
    i += 1


Tâches humaines

In [441]:
roulements_rec = [a for a in Roulements.keys() if 'WPY_REC' in Roulements[a]['Connaissance']]
roulements_for = [a for a in Roulements.keys() if 'WPY_FOR' in Roulements[a]['Connaissance']]
roulements_dep = [a for a in Roulements.keys() if 'WPY_DEP' in Roulements[a]['Connaissance']]
taches_humaines={'WPY_REC': [],'WPY_FOR': [],'WPY_DEP': []}
for index, row in INSTANCE["Taches humaines"].iterrows():
            taches_humaines[row['Chantier']].append(row['Type de tache humaine'])
taches_rec=taches_humaines['WPY_REC']
taches_for=taches_humaines['WPY_FOR']
taches_dep=taches_humaines['WPY_DEP']

## Modèle mathématique 
On a modélisé notre problème en partant du fait que les machines ne sont disponibles que pour un seul train et dans un seul créneau. \
De ce fait, on introduit les variables *binaires* suivantes : \
\
    - $(deb_{ic})$ : variable binaire indiquant si la machine de débranchement est utilisée \
    pour le sillion d'arrivée d'identifiant $i$ au créneau $c$ \
    - $(for_{ic})$ : variable binaire indiquant si la machine de formation est utilisée \
    pour le sillion du départ d'identifiant $i$ au créneau $c$ \
    - $(deg_{ic})$ : variable binaire indiquant si la machine de dégarage est utilisée \
    pour le sillion du départ d'identifiant $i$ au créneau $c$

Les créneaux ont une durée de $15 min$ correspondant à la durée d'une tâche machine. Les valeurs possibles des créneaux dépendent de l'instance, \
soit alors $c \in \{1,2,..., c_{max}\} $ où $c=1$ correspond au premier créneau du lundi de la première semaine du travail : $[0:00 , 0:15[$ et \
$c_{max}$ le dernier créneau du dernier jour du travail : $[23:45, 0:00[$.

On note ainsi : 
$$A = \{\text{Les identifiants des trains d'arrivée}\}$$
$$D = \{\text{Les identifiants des trains du départ}\}$$
$$C= \{1,2,..., c_{max}\}$$

In [442]:
# IMPLEMNTATION DES VARIABLES 
# Implémentation Python
m= Model('Modèle')
deb = {(i, c) : m.addVar(vtype = GRB.BINARY, name=f'deb_{i}_{c}') for i in trains_arrivee for c in range(1,nombre_creneaux+1)}
form = {(i, c) : m.addVar(vtype = GRB.BINARY, name=f'form_{i}_{c}') for i in trains_depart for c in range(1,nombre_creneaux+1)}
deg = {(i, c) : m.addVar(vtype = GRB.BINARY, name=f'deg_{i}_{c}') for i in trains_depart for c in range(1,nombre_creneaux+1)}

Pour inclure les informations sur les voies, nous avons défini les variables binaires suivantes:\
\
    - $orec_{ic}$ : variable binaire indiquant si le train d'arrivée $i \in A$ occupe une voie dans le chantier de débranchement au créneau $c \in C$.\
    - $ofor_{ic}$ : variable binaire indiquant si le train de départ $i \in D$ occupe une voie dans le chantier de formation au créneau $c \in C$.\
    - $odep_{ic}$ : variable binaire indiquant si le train de départ $i \in D$ occupe une voie dans le chantier de départ au créneau $c \in C$.
Les créneaux de $C$, et les ensembles $A$ et $D$ sont définis comme dans le jalon précédent.

In [443]:
orec = {(i, c) : m.addVar(vtype = GRB.BINARY, name=f'orec_{i}_{c}') for i in trains_arrivee for c in range(1,nombre_creneaux+1)}
oform = {(i, c) : m.addVar(vtype = GRB.BINARY, name=f'oform_{i}_{c}') for i in trains_depart for c in range(1,nombre_creneaux+1)}
odep = {(i, c) : m.addVar(vtype = GRB.BINARY, name=f'odep_{i}_{c}') for i in trains_depart for c in range(1,nombre_creneaux+1)}

Pour les tâches humaines, on définit des variables binaires de type $ \text{tache-humaine}_{t,r,i,c}$ où $t$ est un train de départ ou d'arrivée, $r$ est un roulement agent, $i$ est l'indice d'un agent dans ce roulement, et $c \in C$. Où cette variable indique que pendant le créneau $c$, le $i^{ème}$ agent du roulement $r$ travaille sur le train $t$ sur la tâche humaine indiquée.

In [444]:
def fusion_vals(d : dict) -> list:
    L = list()
    for _, vals in d.items():
        L += vals
    return L
#pour rassembler tts les créneaux d'un (r,i) dans une liste
print(nombre_creneaux)

192


In [445]:
#TACHES REC
taches_hum_rec = {(tache,train,roulement,agent,c) : m.addVar(vtype = GRB.BINARY, name=f'{tache},{train},{roulement},{agent}_{c}') for tache in taches_rec for train in trains_arrivee for roulement in roulements_rec for agent in range(1,Roulements[roulement]['Nb agents']+1) for c in range(1,nombre_creneaux+1)}
#TACHES FOR
taches_hum_for = {(tache,train,roulement,agent,c) : m.addVar(vtype = GRB.BINARY, name=f'{tache}_{train},{roulement},{agent}_{c}') for tache in taches_for for train in trains_depart for roulement in roulements_for for agent in range(1,Roulements[roulement]['Nb agents']+1) for c in range(1,nombre_creneaux+1)}
#TACHES DEP
taches_hum_dep = {(tache,train,roulement,agent,c) : m.addVar(vtype = GRB.BINARY, name=f'{tache}_{train},{roulement},{agent}_{c}') for tache in taches_dep for train in trains_depart for roulement in roulements_dep for agent in range(1,Roulements[roulement]['Nb agents']+1) for c in range(1,nombre_creneaux+1)}

## JALON 1

### 1. Unique débranchement, formation et dégarage
Chaque train d'arrivé subit un seul et unique débranchement, chaque train du départ subit un seul et unique débranchement et dégarage. 
$$ \forall i \in A \quad \sum_{c} deb_{i,c} = 1 $$
$$ \forall i \in D \quad \sum_{c} for_{i,c} = 1 $$
$$ \forall i \in D \quad \sum_{c} deg_{i,c} = 1 $$

In [446]:
uniqueDeb_train = {i : m.addConstr(quicksum([deb[(i, c)] for c in range(1, nombre_creneaux + 1)]) ==1, name = f'UniqueDeb_train{i}') for i in trains_arrivee}
uniqueform_train = {i : m.addConstr(quicksum([form[(i, c)] for c in range(1, nombre_creneaux + 1)]) ==1, name = f'UniqueFor_train{i}') for i in trains_depart}
uniqueDeg_train = {i : m.addConstr(quicksum([deg[(i, c)] for c in range(1, nombre_creneaux + 1)]) ==1, name = f'UniqueDeg_train{i}') for i in trains_depart}

### 2. Une seule tâche machine par créneau  
Chaque machine est exploitée par au plus un seul train sur chaque créneau.
$$ \forall c \in C \quad \sum_{i \in A} deb_{i,c} \leq 1 $$
$$ \forall c \in C \quad \sum_{i \in D} for_{i,c} \leq 1 $$
$$ \forall c \in C \quad \sum_{i \in D} deg_{i,c} \leq 1 $$

In [447]:
uniqueDeb_creneau = {c : m.addConstr(quicksum([deb[(i, c)] for i in trains_arrivee]) <=1, name = f'UniqueDeb_creneau{c}') for c in range(1, nombre_creneaux + 1)}
uniqueFor_creneau = {c : m.addConstr(quicksum([form[(i, c)] for i in trains_depart]) <=1, name = f'UniqueFor_creneau{c}') for c in range(1, nombre_creneaux + 1)}
uniqueDeg_creneau = {c : m.addConstr(quicksum([deg[(i, c)] for i in trains_depart ]) <=1, name = f'UniqueDeg_creneau{c}') for c in range(1, nombre_creneaux + 1)}

### 3. Débranchement se fait après l'arrivée du train
Pour chaque train d'arrivée $i$, le débranchement ne peut pas avoir lieu dans les créneaux précedent celui d'arrivée $c^{a}_{i}$ après une heure de tâches humaines.
$$ \forall i \in A \quad \sum_{c \space \leq \space c^{a}_{i}+4} deb_{i,c} = 0$$ 

In [448]:
deb_apres_arrivee = {i : m.addConstr(quicksum([deb[(i, c)] for c in range(1, creneau_arrivee[i] + 5)]) ==0, name = f'deb_apres_arrivee{i}') for i in trains_arrivee}

### 4. Formation d'un train de départ 
Pour chaque train du départ $t$, la formation ne peut avoir lieu qu'après débranchement de tous les trains ayant les wagons qui le constituent. \
On note : $$ \forall t \in D \quad A_t = \{\text{Les identifiants des trains d'arrivée ayant au moins un wagon du train t}\}$$
On peut exprimer la contrainte comme suit : 
$$ \forall t \in D, \space\forall c \in C, \space\forall i \in A_t \quad for_{t,c} \leq \sum_{c' \space \lt \space c} deb_{i,c'}$$ 

In [449]:
for_apres_deb = {(t,c,i) : m.addConstr(form[(t,c)]<=quicksum([deb[(i, c_p)] for c_p in range(1, c)]), name = f'for_apres_deb{t}{c}{i}') for t in trains_depart for c in range(1,nombre_creneaux+1) for i in At[t]}

### 5. Dégarage après formation
Pour chaque train du départ, le dégarage ne peut avoir lieu qu'après la formation du train et $150$ minutes soit $10$ créneaux de tâches humaines.
$$ \forall t \in D, \space\forall c \in C \quad deg_{t,c} \leq \sum_{c' \space \lt \space c-10} for_{t,c'}$$ 

In [450]:
deg_apres_for = {(t,c) : m.addConstr(deg[(t,c)]<=quicksum([form[(t, c_p)] for c_p in range(1, c-9)]), name = f'deg_apres_for{t},{c}') for t in trains_depart for c in range(1,nombre_creneaux+1)}

### 6. Dégarage avant départ
Pour chaque train du départ $t$, le dégarage ne peut pas avoir lieu dans les créneaux suivants \
celui de $20$ minutes (de tâches humaines) avant l'instant du départ qu'on note $h^{d}_{t}$.\
Puisque $20$ minutes n'est pas une durée divisible par $15$, on définit le créneau précis $cp^{d}_{t}$ comme suit:
$$cp^{d}_{t} : \text{\space créneau de l'instant \space} h^{d}_{t}-20\space min$$
$$ \forall t \in D \quad \sum_{ \space c \geq \space cp^{d}_{t}} deg_{t,c} = 0$$ 

In [451]:
creneau_precis={} # dict des paires (train: créneau précis)
for index, row in INSTANCE["Sillons depart"].iterrows():
    string=row["JDEP"]
    if type(row["JDEP"])==pd._libs.tslibs.timestamps.Timestamp:
        string=convToStr(row["JDEP"])
    date=list(map(int,string.split("/")))
    hour=list(map(int,str(row["HDEP"]).split(":")))
    date_precis=datetime.datetime(year=date[2], month=date[1], day=date[0], hour=hour[0], minute=hour[1])-datetime.timedelta(minutes=20)
    creneau_precis[row["n°TRAIN"],convToStr(row["JDEP"])]=int((date_precis-date0).total_seconds()/(60*15))+1

deg_avant_depart = {t : m.addConstr(quicksum([deg[(t, c)] for c in range(creneau_depart[t]-2, nombre_creneaux+1)])==0, name = f'depart_apres_deg{t}') for t in trains_depart}

### 7. Disponibilités des machines
Il y a des créneaux où les machines sont indisponible, soient alors  $ \space C_{deb}, C_{for}, C_{deg} \space$  ces crénaux d'indisponibilités.
$$ \forall i \in A \quad \sum_{c \space \in \space C_{deb}} deb_{i,c} = 0 $$
$$ \forall i \in D \quad \sum_{c \space \in \space C_{for}} for_{i,c} = 0 $$
$$ \forall i \in D \quad \sum_{c \space \in \space C_{deg}} deg_{i,c} = 0 $$

In [452]:
indisponibilite_deb = {i : m.addConstr(quicksum([deb[(i, c)] for c in DEB_INDIS]) ==0, name = f'indisponibilité_deb{i}') for i in trains_arrivee }
indisponibilite_for = {i : m.addConstr(quicksum([form[(i, c)] for c in FOR_INDIS]) ==0, name = f'indisponibilité_for{i}') for i in trains_depart}
indisponibilite_deb = {i : m.addConstr(quicksum([deg[(i, c)] for c in DEG_INDIS]) ==0, name = f'indisponibilité_deg{i}') for i in trains_depart}

## JALON 2

### Contraintes Jalon 2

#### 1- Nombre des voies disponibles
Pour chaque créneau $c \in C$, un chantier ne peut pas accueillir plus de trains que le nombre de voies disponibles: 
$$ \forall c \in C \quad \sum_{i \in A} orec_{i,c} \leq \text{NVREC} $$
$$ \forall c \in C \quad \sum_{i \in D} ofor_{i,c} \leq \text{NVFOR} $$
$$ \forall c \in C \quad \sum_{i \in D} odep_{i,c} \leq \text{NVDEP} $$

Où NVDEB, NVFOR, NVDEG correspondent aux nombres de voies disponibles pour chaque chantiers REC, FOR,
DEP respectivement.

In [453]:
Nb_voies_rec = {c : m.addConstr(quicksum([orec[(i, c)] for i in trains_arrivee]) <= NVREC, name = f'nb_voies_rec{c}') for c in range(1,nombre_creneaux+1) }
Nb_voies_for = {c : m.addConstr(quicksum([oform[(i, c)] for i in trains_depart]) <= NVFOR, name = f'nb_voies_form{c}') for c in range(1,nombre_creneaux+1) }
Nb_voies_dep = {c : m.addConstr(quicksum([odep[(i, c)] for i in trains_depart]) <= NVDEP, name = f'nb_voies_dep{c}') for c in range(1,nombre_creneaux+1) }

#### 2- Indisponibilié des chantiers
Il y a des créneaux dans lesquels les chantiers sont indisponibles qu'on note $C_{rec}, C_{for}, C_{dep}$. 
On interdit alors l'occupation lors de ces créneaux : 

$$ \forall i \in A \quad \sum_{c \space \in \space C_{rec}} orec_{i,c} = 0 $$
$$ \forall i \in D \quad \sum_{c \space \in \space C_{for}} ofor_{i,c} = 0 $$
$$ \forall i \in D \quad \sum_{c \space \in \space C_{dep}} odep_{i,c} = 0 $$

In [454]:
indisponibilite_orec = {i : m.addConstr(quicksum([orec[(i, c)] for c in REC_INDIS]) ==0, name = f'indisponibilité_rec{i}') for i in trains_arrivee }
indisponibilite_oform = {i : m.addConstr(quicksum([oform[(i, c)] for c in OFORM_INDIS]) ==0, name = f'indisponibilité_oform{i}') for i in trains_depart}
indisponibilite_dep = {i : m.addConstr(quicksum([odep[(i, c)] for c in DEP_INDIS]) ==0, name = f'indisponibilité_dep{i}') for i in trains_depart}

#### 3- Occupation du chantier de la réception pour un train d’arrivée
Lors de l'arrivée d'un train $i \in A$, il occupe le chantier de réception dans les créneaux entre celui d'arrivéee $c_a^{i}$ 
et celui de son débranchement $c_{deb}^{i} = \sum_{c' \in C}{c'\times deb_{i,c'}}$. Mathématiquement : 
$$ \forall c \in C, \forall i \in A \quad \left(c \ge c_a^{i}\right) \land \left(c \le c_{deb}^{i}\right) \Rightarrow orec_{i,c}=1$$
$$ M \times orec_{i,c} \ge \left(\varepsilon+c-c_a^{i}\right)\left(c_{deb}^{i}-c+\varepsilon \right)$$
Où $M$ est un majorant suffisament grand et $\varepsilon$ un minorant assez petit.

In [455]:
M=(nombre_creneaux)**2
eps=0.1
creneau_deb={i : quicksum([u*deb[(i,u)] for u in range(1,nombre_creneaux+1)]) for i in trains_arrivee}
occupation_arrivee_deb={(i,c) : m.addConstr((eps+(c-creneau_arrivee[i]))*(creneau_deb[i]-c+eps)<=M*orec[(i,c)], name=f'occupation_arrivee_deg_{i,c}') for i in trains_arrivee for c in range(1,1+nombre_creneaux)}

#### 4- Lien entre débranchement d’un train d’arrivée et occupation de voies dans le chantier de formation
Pour un train d'arrivée $t \in A$, la machine de débranchement le débranche et renvoie ses wagones vers le chantier de formation dans le même créneau. 
On définit pour un train d'arrivée $t \in A$, l'ensemble $D_t$ des trains de départ associés aux wagons de $t$. 
$$ \forall t \in A \quad D_t = \{\text{Les identifiants des trains du départ ayant au moins un wagon du train t}\}$$
Il faut alors garantir: 
$$
    \forall t \in A, \forall c \in C \quad deb_{t,c}=1 \Rightarrow \prod\limits_{i \in D_t} ofor_{i,c} =1
$$
qu'on linéarise comme suit:
$$
    \forall t \in A, \forall i \in D_t, \forall c \in C \quad deb_{t,c} \leq ofor_{i,c}
$$

In [456]:
reserve_voies_for_au_deb = {(t,c,i) : m.addConstr(deb[(t,c)]<=oform[(i,c)], name = f'reserve_voies_for_au_deb{t}{c}{i}') for t in trains_arrivee for c in range(1,nombre_creneaux+1) for i in Dt[t]}

#### Occupation du chantier de formation
#### 5-Créneaux d’occupation 
Pour faire une relation de récurrence entre les créneaux d'occupation du chantier de formation pour un train de départ, on dit que si un train de départ occupe le chantier formation pendant un créneau $c$, et que pendant ce créneau $c$ le train ne se dégare pas, alors il reste dans le chantier formation. Mathématiquement , cela se traduit par:  
$$\forall t \in D, \forall c \in C,  ofor_{t,c}=1 \land deg_{t,c}=0 \Rightarrow ofor_{t,c+1}=1$$
Pour linériser cette contrainte, on introduit des variables binaires $z_{t,c}$ symbolisant la conjonction des deux propositions: 
$$z_{t,c} \leq ofor_{t,c}$$
$$z_{t,c} \leq 1-deg_{t,c}$$
$$ z_{t,c} \geq ofor_{t,c}+1-deg_{t,c} -1$$
$$z_{t,c} \leq ofor_{t,c+1}$$

In [457]:
z = {(t, c) : m.addVar(vtype = GRB.BINARY, name=f'z_{t}_{c}') for t in trains_depart for c in range(1,nombre_creneaux)}
duree_occupe_form1={(t,c): m.addConstr(z[(t,c)]<=oform[(t,c)], name = f'duree_occupe_form1{t,c}') for t in trains_depart for c in range(1,nombre_creneaux)}
duree_occupe_form2={(t,c): m.addConstr(z[(t,c)]<=1-deg[(t,c)], name = f'duree_occupe_form2{t,c}') for t in trains_depart for c in range(1,nombre_creneaux)}
duree_occupe_form3={(t,c): m.addConstr(z[(t,c)]>=oform[(t,c)]-deg[(t,c)], name = f'duree_occupe_form3{t,c}') for t in trains_depart for c in range(1,nombre_creneaux)}
duree_occupe_form4={(t,c): m.addConstr(z[(t,c)]<=oform[(t,c+1)], name = f'duree_occupe_form4{t,c}') for t in trains_depart for c in range(1,nombre_creneaux)}

#### 6- Occupation d'une voie dans le chantier formation lors du dégarage
Il est également crucial de s'assurer qu'un train de départ occupe le chantier de formation lors de son opération de dégarage :
\begin{equation}
\forall t \in D, \forall c \in C \quad deg_{t,c} \le ofor_{t,c}
\end{equation}

In [458]:
reserve_voie_for_au_deg = {(t,c) : m.addConstr(deg[(t,c)]<=oform[(t,c)], name = f'reserve_voie_for_au_deg{t}{c}') for t in trains_depart for c in range(1,nombre_creneaux+1)}

#### 7- Créneaux où on libère les voies
Après le dégarage, un train doit libérer le chantier de formation pour permettre la réorganisation des wagons pour d'autres trains. La libération des voies est modélisée par la contrainte suivante :
$$ \forall t \in D, \forall c \in C, c \geq c_{deg}^{t}+1 \Rightarrow  ofor_{t,c}=0$$
Où 
$$ c_{deg}^{t}= \sum_{u}{u\times deg_{t,u}} $$
Linéarisée avec: 
$$(c-c_{deg}^{t}) \leq M \times (1- ofor_{t,c})$$

In [459]:
creneau_deg={i : quicksum([u*deg[(i,u)] for u in range(1,nombre_creneaux+1)]) for i in trains_depart}
non_occupation_for_apres_deg={(t,c) : m.addConstr((c-creneau_deg[t])<=M*(1-oform[(t,c)]), name=f'occupation_for_qd_deg_{t,c}') for t in trains_depart for c in range(1,1+nombre_creneaux)}

#### 8- Occupation du chantier de départ pour un train d'arrivée
Pour chaque train de départ $t \in D$, il occupe une voie dans le chantier de départ entre le moment de son dégarage ($\sum_{c' \in C}{c'\times deg_{t,c'}}$) et son départ prévu ($c_d^{t}$). Mathématiquement, cela est représenté par :

$$\forall c \in C, \forall t \in D \quad \left(c \le c_d^{t}\right) \land \left(c \ge \sum_{c' \in C}{c'\times deg_{t,c'}}\right) \Rightarrow odep_{t,c}=1$$
On occupe une voie du départ après dégarage et avant le départ du train:
$$ M \times odep_{t,c} \ge \left(\varepsilon+(c_d^{i}-c)\right)\left(c-\sum_{u}{u\times deg_{t,u}}+\varepsilon \right)$$

In [460]:
M=(nombre_creneaux)**2
eps=0.1
occupation_deg_depart={(i,c) : m.addConstr((eps+creneau_depart[i]-c)*(c-creneau_deg[i]+eps)<=M*odep[(i,c)], name=f'occupation_deg_depart_{i,c}') for i in trains_depart for c in range(1,1+nombre_creneaux)}


## JALON 3

 ### Variables auxiliaires
 #### $\text{Tache-humaine}_{t,c}$
 Pour raisonner comme pendant le jalon 1 sur chaque train $t$ et créneau $c$ pour savoir si la tâche humaine s'effectue sur le train $t$ au créneau $c$, on peut retrouver ça avec: 
 $$\text{tâche-humaine}_{t,c} = \sum_{\text{agents} (r,i)} \text{tâche-humaine}_{t,r,i,c}$$

In [461]:
taches_hum_arr_tc = {(tache,train,c) : quicksum([taches_hum_rec[(tache,train,roulement,i,c)] for roulement in roulements_rec for i in range(1,Roulements[roulement]['Nb agents']+1)])   for train in trains_arrivee  for tache in taches_rec for c in range(1,nombre_creneaux+1)}
taches_hum_for_tc = {(tache,train,c) : quicksum([taches_hum_for[(tache,train,roulement,i,c)] for roulement in roulements_for for i in range(1,Roulements[roulement]['Nb agents']+1)])   for train in trains_depart  for tache in taches_for for c in range(1,nombre_creneaux+1)}
taches_hum_dep_tc = {(tache,train,c) : quicksum([taches_hum_dep[(tache,train,roulement,i,c)] for roulement in roulements_dep for i in range(1,Roulements[roulement]['Nb agents']+1)])   for train in trains_depart  for tache in taches_dep for c in range(1,nombre_creneaux+1)}

#### Nombre de tâches effectuées par un agent
De plus chaque agent i appartient à un roulement r, on définit donc l'ensemble qui associe à chaque couple (roulement,agent), les blocs où chaque agent pour travailler, par exemple pour un roulement r et un agent i, dans une journée $j$ où cet agent peut travailler, il peut travailler dans un des blocs:  $$bloc_{r,i,j} =  [(00:00-08:00),(8:00-16:00),\cdots]$$
 
 Ainsi, dans un bloc de travail,  $  \forall  bloc  \in  bloc_{r,i,j},   $ le nombre des taches effectuées par l'agent (r,i) ($i^{ème}$ agent du roulement $r$) dans ce bloc est: 
 $$\forall \text{jour}, \forall \text{bloc} \in blocs_{r,i,jour}, \quad \text{nb-taches}_{(r,i),bloc,jour} = \sum_{c \in bloc} \quad  \sum_{\text{taches faisables par (r,i)}} \quad \sum_{t \in \text{trains de la tache}} \text{tâche-humaine}_{t,r,i,c} $$

In [462]:
nb_taches_rec={(r,i,b): quicksum([taches_hum_rec[(tache,t,r,i,c)] for tache in taches_rec for t in trains_arrivee for c in Roulements[r]['Creneaux'][b]])   for r in roulements_rec for i in range(1,Roulements[r]['Nb agents']+1) for b in Roulements[r]['Creneaux']}
nb_taches_for={(r,i,b): quicksum([taches_hum_for[(tache,t,r,i,c)] for tache in taches_for for t in trains_depart for c in Roulements[r]['Creneaux'][b]])   for r in roulements_for for i in range(1,Roulements[r]['Nb agents']+1) for b in Roulements[r]['Creneaux']}
nb_taches_dep={(r,i,b): quicksum([taches_hum_dep[(tache,t,r,i,c)] for tache in taches_dep for t in trains_depart for c in Roulements[r]['Creneaux'][b]])   for r in roulements_dep for i in range(1,Roulements[r]['Nb agents']+1) for b in Roulements[r]['Creneaux']}

#certaines tâches vont se compter plusieurs fois (tâches nécessitant Dt>c)...

def fusion_dictionnaires(*dicts):
    result = {}
    for d in dicts:
        for key, value in d.items():
            result[key] = result.get(key, 0) + value
    return result
nb_taches=fusion_dictionnaires(nb_taches_rec,nb_taches_for,nb_taches_dep)
#keys de nb_taches sont (roulement,agent,bloc) où bloc est de type (jour,bloc)

### Contraintes

#### Un agent travaille pendant un bloc au plus pendant une journée de service:
Pour une journée de service, chaque agent (r,i) peut travailler dans un bloc horaire au plus, on impose donc la contrainte suivante:\
Sur une journée de service, si le nombre de tâches effectuées par l'agent dans 1 de ses blocs possibles de travail est non nul, il faut que le nombre de tâches effectuées dans les autres blocs possibles soit nul. 
Donc, si par exemple , dans une journée de service , on a trois blocs de travail possibles pour l'agent (r,i), qui sont $J_1, J_2$ et $J_3$.On a:
$$ \text{nb-taches}_{(r,i),J1} \neq 0 \quad \Rightarrow \quad \text{nb-taches}_{(r,i),J_2}=\text{nb-taches}_{(r,i),J_3}=0$$
Et de même par permutation des blocs. Il faut alors garantir, pour l'agent (r,i):
$$ \forall c \in J_1: \exists \text{train} t, \text{tâche}, \quad \text{t.q}  \quad \text{tâche}_{t,r,i,c}=1 \Rightarrow \quad \text{nb-taches}_{(r,i),J2}+\text{nb-taches}_{(r,i),J3}=0$$
Qu'on linéarise par:
$$\forall \text{agent} (r,i), \forall \text{tâche}, \forall \text{train},\forall c \in J_1: \quad   \text{nb-taches}_{(r,i),J2}+\text{nb-taches}_{(r,i),J3} \leq M \times (1-\text{tâche}_{t,r,i,c})$$

In [463]:
#Pour un agent (r,i), il faut extraire les blocs de chaque jour et puis faire les contraintes sur ces blocs:
def extraire_meme_jour(dictionnaire):
    """
    Cette fonction prend en entrée un dictionnaire dont les clés sont de la forme (roulement, agent, (jour, bloc horaire)).
    Elle extrait les couples (jour, bloc horaire) qui correspondent au même jour pour chaque (roulement, agent).
    La sortie est un dictionnaire où les clés sont les couples (roulement, agent) et les valeurs sont des dictionnaires
    dont les clés sont les jours et les valeurs sont des tuples (jour, bloc horaire) associés à ces jours pour ce couple.
    """
    result = {}
    for cle, _ in dictionnaire.items():
        roulement, agent, (jour, bloc_horaire) = cle
        cle_roulement_agent = (roulement, agent)
        if cle_roulement_agent in result:
            if jour in result[cle_roulement_agent]:
                result[cle_roulement_agent][jour].append((jour,bloc_horaire))
            else:
                result[cle_roulement_agent][jour] = [(jour,bloc_horaire)]
        else:
            result[cle_roulement_agent] = {jour: [(jour,bloc_horaire)]}

    return result
agents_jours_blocs = extraire_meme_jour(nb_taches)

In [464]:
agent_rec_work_bloc={(agent,jour,tache,train,c,J1): m.addConstr(quicksum([nb_taches[(agent[0],agent[1],J)] for J in agents_jours_blocs[agent][jour] if J!=J1]) <=M*(1-taches_hum_rec[(tache,train,agent[0],agent[1],c)]),name = f'agent_rec_work_bloc_{agent}_{jour}_{tache}_{train}_{c}_{J1}') for agent in agents_jours_blocs if agent[0] in roulements_rec for jour in agents_jours_blocs[agent] for J1 in agents_jours_blocs[agent][jour] for c in Roulements[agent[0]]['Creneaux'][J1] for train in trains_arrivee for tache in taches_rec}
agent_for_work_bloc={(agent,jour,tache,train,c,J1): m.addConstr(quicksum([nb_taches[(agent[0],agent[1],J)] for J in agents_jours_blocs[agent][jour] if J!=J1]) <=M*(1-taches_hum_for[(tache,train,agent[0],agent[1],c)]),name = f'agent_for_work_bloc_{agent}_{jour}_{tache}_{train}_{c}_{J1}') for agent in agents_jours_blocs if agent[0] in roulements_for for jour in agents_jours_blocs[agent] for J1 in agents_jours_blocs[agent][jour] for c in Roulements[agent[0]]['Creneaux'][J1] for train in trains_depart for tache in taches_for}
agent_dep_work_bloc={(agent,jour,tache,train,c,J1): m.addConstr(quicksum([nb_taches[(agent[0],agent[1],J)] for J in agents_jours_blocs[agent][jour] if J!=J1]) <=M*(1-taches_hum_dep[(tache,train,agent[0],agent[1],c)]),name = f'agent_dep_work_bloc_{agent}_{jour}_{tache}_{train}_{c}_{J1}') for agent in agents_jours_blocs if agent[0] in roulements_dep for jour in agents_jours_blocs[agent] for J1 in agents_jours_blocs[agent][jour] for c in Roulements[agent[0]]['Creneaux'][J1] for train in trains_depart for tache in taches_dep}

#### Chaque tache humaine est réalisée par un seul agent et dans un seul créneau:

Afin de réaliser une tache humaine, un agent disponible est suffisant, et pour un train $t$, la tâche doit être réalisée par un agent et dans un créneau, on traduit cette contrainte pour chaque train t :
$$\forall \text{train} \, t, \forall \, \text{tache-humaine}, \sum_{c \in C} \sum_{agents (r,i)} \text{tache-humaine}_{t,r,i,c} = 1$$

In [465]:
tache_realisee_1_agent_rec={(train,tache): m.addConstr(quicksum([taches_hum_rec[(tache,train,agent[0],agent[1],c)] for agent in agents_jours_blocs if agent[0] in roulements_rec for c in range(1,nombre_creneaux+1)])==1,name = f'tache_realisee_1_agent_rec_{train}_{tache}') for train in trains_arrivee for tache in taches_rec}
tache_realisee_1_agent_for={(train,tache): m.addConstr(quicksum([taches_hum_for[(tache,train,agent[0],agent[1],c)] for agent in agents_jours_blocs if agent[0] in roulements_for for c in range(1,nombre_creneaux+1)])==1,name = f'tache_realisee_1_agent_for_{train}_{tache}') for train in trains_depart for tache in taches_for}
tache_realisee_1_agent_dep={(train,tache): m.addConstr(quicksum([taches_hum_dep[(tache,train,agent[0],agent[1],c)] for agent in agents_jours_blocs if agent[0] in roulements_dep for c in range(1,nombre_creneaux+1)])==1,name = f'tache_realisee_1_agent_dep_{train}_{tache}') for train in trains_depart for tache in taches_dep}

In [466]:
print(agents_jours_blocs)

{('roulement2', 1): {'1': [('1', '05:00-13:00'), ('1', '13:00-21:00')], ' 2': [(' 2', '05:00-13:00'), (' 2', '13:00-21:00')], ' 3': [(' 3', '05:00-13:00'), (' 3', '13:00-21:00')], ' 4': [(' 4', '05:00-13:00'), (' 4', '13:00-21:00')], ' 5': [(' 5', '05:00-13:00'), (' 5', '13:00-21:00')], ' 6': [(' 6', '05:00-13:00'), (' 6', '13:00-21:00')], ' 7': [(' 7', '05:00-13:00'), (' 7', '13:00-21:00')]}, ('roulement2', 2): {'1': [('1', '05:00-13:00'), ('1', '13:00-21:00')], ' 2': [(' 2', '05:00-13:00'), (' 2', '13:00-21:00')], ' 3': [(' 3', '05:00-13:00'), (' 3', '13:00-21:00')], ' 4': [(' 4', '05:00-13:00'), (' 4', '13:00-21:00')], ' 5': [(' 5', '05:00-13:00'), (' 5', '13:00-21:00')], ' 6': [(' 6', '05:00-13:00'), (' 6', '13:00-21:00')], ' 7': [(' 7', '05:00-13:00'), (' 7', '13:00-21:00')]}, ('roulement2', 3): {'1': [('1', '05:00-13:00'), ('1', '13:00-21:00')], ' 2': [(' 2', '05:00-13:00'), (' 2', '13:00-21:00')], ' 3': [(' 3', '05:00-13:00'), (' 3', '13:00-21:00')], ' 4': [(' 4', '05:00-13:00')

#### Un agent effectue une tâche humaine au plus par créneau:
$$\forall \text{agent} (r,i), \forall c \in C,  \sum_{\text{taches faisables par l'agent}} \quad \sum_{\text{trains t}} \quad  \text{tache}_{t,r,i,c} \leq 1$$

In [467]:
#On definit comme auparavant le nombre de taches effectués par un agent mais cette fois pendant un seul créneau
nb_taches_rec_c={(r,i,c): quicksum([taches_hum_rec[(tache,t,r,i,c)] for tache in taches_rec for t in trains_arrivee])   for r in roulements_rec for i in range(1,Roulements[r]['Nb agents']+1) for c in range(1, nombre_creneaux+1)}
nb_taches_for_c={(r,i,c): quicksum([taches_hum_for[(tache,t,r,i,c)] for tache in taches_for for t in trains_depart])   for r in roulements_for for i in range(1,Roulements[r]['Nb agents']+1) for c in range(1, nombre_creneaux+1)}
nb_taches_dep_c={(r,i,c): quicksum([taches_hum_dep[(tache,t,r,i,c)] for tache in taches_dep for t in trains_depart])   for r in roulements_dep for i in range(1,Roulements[r]['Nb agents']+1) for c in range(1, nombre_creneaux)}

nb_taches_c=fusion_dictionnaires(nb_taches_rec_c,nb_taches_for_c,nb_taches_dep_c)

une_tache_au_plus_par_c={(r,i,c): m.addConstr(nb_taches_c[(r,i,c)]<=1 ,name = f'une_tache_au_plus_par_c_{r}_{i}_{c}') for r,i,c in nb_taches_c.keys()}

In [468]:
print(len(nb_taches_rec_c.keys()))

1536


#### Respect de l'ordre des taches humaines
On définit des contraintes sur l'ordre d'exécution des taches humaines et on assure que certaines taches humaines se font en parallèle aux taches machines. \
$\textbf{Ordre séquentiel entre les tâches:}$\
Un exemple d'ordre séquentiel d'excution des tâches humaines est la contrainte que la tâche $\text{arrivee-reception}$ vient avant $\text{preparation-tri}$, pour les trains d'arrivée. $t \in A$, on écrit cette contrainte comme suit :
$$\forall t \in A : \sum_{c \in C} c \times \text{arrivee-reception}_{t,c} < \sum_{c \in C} c \times \text{preparation-tri}_{t,c} $$
Cela est fait pour toutes les tâches ordonnées séquentiellement décrites dans la feuille des tâches humaines.
Il faut aussi garantir que la tâche humaine de réception se fait après l'arrivée d'un train d'arrivée et que son essai de frein départ a lieu avant son départ.

In [469]:
#REC
arrivee_avant_rec={t: m.addConstr(quicksum([c*taches_hum_arr_tc[('arrivée Reception',t,c)] for c in range(1,nombre_creneaux+1)])>=creneau_arrivee[t],name = f'arrivee_avant_rec_{t}') for t in trains_arrivee}
rec_avant_tri={t: m.addConstr(quicksum([c*taches_hum_arr_tc[('arrivée Reception',t,c)] for c in range(1,nombre_creneaux+1)])<=quicksum([c*taches_hum_arr_tc[('préparation tri',t,c)] for c in range(1,nombre_creneaux+1)])-1,name = f'rec_avant_tri_{t}') for t in trains_arrivee}
tri_avant_deb={t: m.addConstr(quicksum([c*taches_hum_arr_tc[('préparation tri',t,c)] for c in range(1,nombre_creneaux+1)])<=quicksum([c*taches_hum_arr_tc[('débranchement',t,c)] for c in range(1,nombre_creneaux+1)])-1,name = f'tri_avant_deb_{t}') for t in trains_arrivee}

#FOR
appui_avant_attelage={t: m.addConstr(quicksum([c*taches_hum_for_tc[('appui voie + mise en place câle',t,c)] for c in range(1,nombre_creneaux+1)])<=quicksum([c*taches_hum_for_tc[('attelage véhicules',t,c)] for c in range(1,nombre_creneaux+1)])-1,name = f'appui_avant_atelage_{t}') for t in trains_depart}
attelage_avant_deg={t: m.addConstr(quicksum([c*taches_hum_for_tc[('attelage véhicules',t,c)] for c in range(1,nombre_creneaux+1)])<=quicksum([c*taches_hum_for_tc[('dégarage / bouger de rame',t,c)] for c in range(1,nombre_creneaux+1)])-1,name = f'attelage_avant_deg_{t}') for t in trains_depart}
#DEP
deg_avant_essai_frein={t: m.addConstr(quicksum([c*taches_hum_for_tc[('dégarage / bouger de rame',t,c)] for c in range(1,nombre_creneaux+1)])<=quicksum([c*taches_hum_dep_tc[('essai de frein départ',t,c)] for c in range(1,nombre_creneaux+1)])-1,name = f'deg_avant_essai_frein_{t}') for t in trains_depart}



$\textbf{Parallélisation de tâches/ Couplage tâches humaines et tâches machines }$\
On couple les variables des tâches machines définies au jalon 1 avec les variables des tâches humaines qui vont parallèlement avec ces tâches machines:
$$\forall t \in A, \forall c \in C , \text{debranchement}_{t,c} = deb_{t,c}$$
$$\forall t \in D, \forall c \in C , \text{appui voie + mise en place câle}_{t,c} = for_{t,c}$$
$$\forall t \in D, \forall c \in C , \text{dégarage / bouger de rame}_{t,c} = deg_{t,c}$$

In [470]:
debranchement_deb={(t,c): m.addConstr(taches_hum_arr_tc[('débranchement',t,c)]==deb[(t,c)],name = f'debranchement_deb_{t}') for t in trains_arrivee for c in range(1,nombre_creneaux+1)}
appui_for={(t,c): m.addConstr(taches_hum_for_tc[('appui voie + mise en place câle',t,c)]==form[(t,c)],name = f'appui_for_{t}') for t in trains_depart for c in range(1,nombre_creneaux+1)}
bouger_deg={(t,c): m.addConstr(taches_hum_for_tc[('appui voie + mise en place câle',t,c)]==deg[(t,c)],name = f'bouger_deg_{t}') for t in trains_depart for c in range(1,nombre_creneaux+1)}

#### Indisponibilités des chantiers
Dans chaque créneau d'indisponibilité du chantier de réception, le nombre de tâches dans ce chantier effectuées par chaque agent est nul:
$$\forall \text{agent} (r,i), \sum_{c \in C_rec} \text{nb-taches-rec}_{r,i,c}=0 $$
$$\forall \text{agent} (r,i), \sum_{c \in C_formation} \text{nb-taches-for}_{r,i,c}=0 $$
$$\forall \text{agent} (r,i), \sum_{c \in C_dep} \text{nb-taches-dep}_{r,i,c}=0 $$

In [471]:
indispo_rec_hum={(r,i): m.addConstr(quicksum([nb_taches_rec_c[(r,i,c)] for c in REC_INDIS])==0,name = f'indispo_rec_hum={r}_{i}') for r in roulements_rec for i in range(1,Roulements[r]['Nb agents']+1)}
indispo_for_hum={(r,i): m.addConstr(quicksum([nb_taches_for_c[(r,i,c)] for c in OFORM_INDIS])==0,name = f'indispo_for_hum={r}_{i}') for r in roulements_for for i in range(1,Roulements[r]['Nb agents']+1)}
indispo_dep_hum={(r,i): m.addConstr(quicksum([nb_taches_dep_c[(r,i,c)] for c in DEP_INDIS])==0,name = f'indispo_dep_hum={r}_{i}') for r in roulements_dep for i in range(1,Roulements[r]['Nb agents']+1)}

#### Un agent ne travaille que pendant ses blocs horaires
Cela s'exprime comme suit:
$$\forall \text{agent} (r,i): \quad \sum_{j \in jours}\sum_{bloc \in \text{dispo}_{(r,i),j}} \text{nb-taches}_{(r,i),bloc} >= \sum_{c \in C} \text{nb-taches-c}_{(r,i),c} $$

In [472]:
def indispo(L : list, j : int) -> list :
    U = list(range((j-1)*96+1, j*96+1))
    return [a for a in U if a not in L and a<nombre_creneaux+1]

def val_fusion_jour(d : dict, j : int):
    L = list()
    for key, val in d.items():
        if j == int(key[0]) :
            L += val
    return L
indispo_agent = {(r,i): m.addConstr(quicksum([nb_taches_c[(r,i,c)] for c in range(1, nombre_creneaux+1)]) >= quicksum([nb_taches[(r,i,b)] for b in Roulements[r]['Creneaux']]),name = f'indispo_agent_{r}_{i}') for r in Roulements for i in range(1,Roulements[r]['Nb agents']+1)}

In [473]:
print(len(nb_taches_c.keys())//len(nb_taches.keys()))

12


In [474]:
print(indispo(val_fusion_jour(Roulements['roulement1']['Creneaux'],2),2))
print(Roulements['roulement1']['Creneaux'])


[97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116]
{('1', '05:00-13:00'): [21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52], ('1', '13:00-21:00'): [53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84], ('1', '21:00-05:00'): [85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116], (' 2', '05:00-13:00'): [117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148], (' 2', '13:00-21:00'): [149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180], (' 2', '21:00-05:00'): [181, 182, 183, 184, 185, 186, 187,

## Résolution

#### Objectif Jalon 3
Pour un agent $(r,i)$ , dans un jour $j$, le nombre de tâches effectuées par l'agent dans ce jour est:
$$ \text{nb-taches}_{(r,i),j}= \sum_{\text{blocs du jour j}} \text{nb-tache}_{(r,i),bloc}$$

On définit ainsi des variables binaires:
$$\text{travaille}_{(r,i),j}=\begin{cases} 
1 & \text{si au créneau c, on affecte la tache humaine à l'agent i du roulement r sur le train t.} \\
0 & \text{sinon.}
\end{cases}$$

Ces variables doivent alors vérifier la contrainte:
$$\forall \text{agent} (r,i), \forall \text{jour} j: \text{nb-taches}_{(r,i),j} \gt 0 \Rightarrow \text{travaille}_{(r,i),j}=1$$

Qu'on linéarise par:
$$\text{nb-taches}_{(r,i),j} \leq M \times \text{travaille}_{(r,i),j}$$

In [475]:
nb_taches_jour={(r,i,j): quicksum([nb_taches[(r,i,b)] for b in Roulements[r]['Creneaux']]) for r,i,_ in nb_taches.keys() for j in Roulements[r]['Jours']}
 

In [476]:
travaille_jour={(r,i,j): m.addVar(vtype = GRB.BINARY, name=f'travaille_{r},{i}_{j}') for r,i,_ in nb_taches.keys() for j in Roulements[r]['Jours'] }

In [477]:
contrainte_travaille={(r,i,j): m.addConstr(nb_taches_jour[(r,i,j)]<=travaille_jour[(r,i,j)]*M,name = f'contrainte_travaille_{r}_{i}_{j}') for r,i,_ in nb_taches.keys() for j in Roulements[r]['Jours'] }

#### Fonction objectif jalon 3: 
Pour chaque agent $(r,i)$ on veut minimiser le nombre de jours dans lesquels celui-ci travaille. Ainsi, on prend comme fonction objectif:
$$\sum_{\text{agents (r,i)}} \quad \sum_{\text{jours $j$}} \text{travaille}_{r,i,j} $$

In [478]:
objfunction_jalon3=quicksum([travaille_jour[(r,i,j)] for r,i,_ in nb_taches.keys() for j in Roulements[r]['Jours']])

In [479]:
objfunction_for = quicksum([oform[(t,c)] for t in trains_depart for c in range(1,nombre_creneaux+1)])
objfunction_dep = quicksum([odep[(t,c)] for t in trains_depart for c in range(1,nombre_creneaux+1)])
objfunction_rec = quicksum([orec[(t,c)] for t in trains_arrivee for c in range(1,nombre_creneaux+1)])
m.setObjective(objfunction_for+objfunction_dep+objfunction_rec+objfunction_jalon3, GRB.MINIMIZE)
m.params.outputflag = 0
m.update()

In [480]:
m.optimize()
if m.status == GRB.INF_OR_UNBD:
    m.setParam(GRB.Param.Presolve, 0)
    m.optimize()

if m.status == GRB.INFEASIBLE:
    print("\n\tN'A PAS DE SOLUTION!!!")
    m.computeIIS()
    for c in m.getConstrs():
        if c.IISConstr:
            print(c.ConstrName)
elif m.status == GRB.UNBOUNDED:
    print("\n\tEST NON BORNÉ!!!")


	N'A PAS DE SOLUTION!!!
UniqueFor_train('sillon4', '02/05/2023')
UniqueDeg_train('sillon4', '02/05/2023')
deb_apres_arrivee('sillon2', '02/05/2023')
deb_apres_arrivee('sillon3', '02/05/2023')
for_apres_deb('sillon4', '02/05/2023')3('sillon2', '02/05/2023')
for_apres_deb('sillon4', '02/05/2023')4('sillon2', '02/05/2023')
for_apres_deb('sillon4', '02/05/2023')5('sillon2', '02/05/2023')
for_apres_deb('sillon4', '02/05/2023')8('sillon3', '02/05/2023')
for_apres_deb('sillon4', '02/05/2023')9('sillon2', '02/05/2023')
for_apres_deb('sillon4', '02/05/2023')10('sillon3', '02/05/2023')
for_apres_deb('sillon4', '02/05/2023')11('sillon3', '02/05/2023')
for_apres_deb('sillon4', '02/05/2023')12('sillon3', '02/05/2023')
for_apres_deb('sillon4', '02/05/2023')13('sillon2', '02/05/2023')
for_apres_deb('sillon4', '02/05/2023')16('sillon3', '02/05/2023')
for_apres_deb('sillon4', '02/05/2023')18('sillon2', '02/05/2023')
for_apres_deb('sillon4', '02/05/2023')19('sillon2', '02/05/2023')
for_apres_deb('sillo

In [481]:
try:
    m.optimize()
except GRB.GurobiError as e:
    if e.errno == 10009:  # Gurobi out of memory error
        print("Out of memory error occurred. Retrieving the last solution found.")
    else:
        raise e

# Accéder à la dernière solution trouvée
if m.Status == GRB.Status.OPTIMAL or m.Status == GRB.Status.SUBOPTIMAL:
    for v in m.getVars():
        print('%s %g' % (v.VarName, v.X))
    print('Objective: %g' % m.ObjVal)
else:
    print('Aucune solution optimale trouvée')


Aucune solution optimale trouvée


In [482]:
outputxlsx = "output.xlsx"
output = {'Id tâche': [], 'Type de tâche' : [], 'Jour' : [], 'Arr/Dep' : [],'Heure début' : [], 'Durée' : [], 'Sillon' : [],
          'Occupation des voies par chantier (optim)': [], 'WPY_REC' : [], 'WPY_FOR' : [], 'WPY_DEP' : []}
nb_voie_occ={"WPY_REC": [],
             "WPY_FOR": [],
             "WPY_DEP": []}
for c in range(1,1+nombre_creneaux):
    # Occupation des voies
    nb_voie_occ["WPY_REC"].append(sum([orec[(t,c)].x for t in trains_arrivee]))
    nb_voie_occ["WPY_FOR"].append(sum([oform[(t,c)].x for t in trains_depart]))
    nb_voie_occ["WPY_DEP"].append(sum([odep[(t,c)].x for t in trains_depart]))
    
    # Tâches machines sur les trains d'arrivee 
    for train in trains_arrivee:
        if deb[(train,c)].x==1:
            output['Id tâche'].append(f'DEB_{train[0]}_'+ creneau_to_date(date0, c).strftime('%d/%m/%Y'))
            output['Type de tâche'].append('DEB')
            output['Jour'].append(creneau_to_date(date0, c).strftime('%d/%m/%Y'))
            output['Arr/Dep'].append(creneau_to_date(date0, creneau_arrivee[train]))
            output['Heure début'].append(creneau_to_date(date0, c).strftime('%H:%M'))
            output['Durée'].append(15)
            output['Sillon'].append(train[0])
    
    # Tâches machines ur les trains du départ
    for train in trains_depart:
        if form[(train,c)].x==1:
            output['Id tâche'].append(f'FOR_{train[0]}_'+ creneau_to_date(date0, c).strftime('%d/%m/%Y'))
            output['Type de tâche'].append('FOR')
            output['Jour'].append(creneau_to_date(date0, c).strftime('%d/%m/%Y'))
            output['Arr/Dep'].append(creneau_to_date(date0, creneau_depart[train]))
            output['Heure début'].append(creneau_to_date(date0, c).strftime('%H:%M'))
            output['Durée'].append(15)
            output['Sillon'].append(train[0])
        if deg[(train,c)].x==1:
            output['Id tâche'].append(f'DEG_{train[0]}_' + creneau_to_date(date0, c).strftime('%d/%m/%Y'))
            output['Type de tâche'].append('DEG')
            output['Jour'].append(creneau_to_date(date0, c).strftime('%d/%m/%Y'))
            output['Arr/Dep'].append(creneau_to_date(date0, creneau_depart[train]))
            output['Heure début'].append(creneau_to_date(date0, c).strftime('%H:%M'))
            output['Durée'].append(15)
            output['Sillon'].append(train[0])

output["Occupation des voies par chantier (optim)"].append("Taux max d'occupation des voies (en %)")
output["Occupation des voies par chantier (optim)"].append("Nombre max de voies occupées")
output["Occupation des voies par chantier (optim)"].append("Nombre total de voies à disposition")
while len(output["Occupation des voies par chantier (optim)"])!=len(output["Id tâche"]):
    output["Occupation des voies par chantier (optim)"].append(None)

nb_voies={"WPY_REC": NVREC,
             "WPY_FOR": NVFOR,
             "WPY_DEP": NVDEP}
for key in nb_voies:    
    output[key].append(100*max(nb_voie_occ[key])/nb_voies[key])
    output[key].append(max(nb_voie_occ[key]))
    output[key].append(nb_voies[key])
    while len(output[key])!=len(output["Id tâche"]):
        output[key].append(None)

# Piping output into an excel sheet
outputdf=pd.DataFrame(output)
outputdf.to_excel(outputxlsx, index=False)

AttributeError: Unable to retrieve attribute 'x'

In [ ]:
for train in trains_arrivee:
    print('-----------------------------------------------------------------')
    print('Pour le train d arrivee: ',train)
    print('Arrivee a ', creneau_arrivee[train])
    creneaux_occup_rec=[]
    for c in range(1,nombre_creneaux+1):
        if deb[(train,c)].x==1:
            print('deb de {} se fait au créneau {}:'.format(train,c))
        if orec[(train,c)].x==1:
            creneaux_occup_rec.append(c)
    print('Il occupe le chantier reception dans les horaires: ',creneaux_occup_rec)
    print('Ce train est lié aux trains de départ: ', Dt[train])
    print('-------------------------------------------------------')


for train in trains_depart:
    creneaux_occup_for=[]
    creneaux_occup_dep=[]
    print('--------------------------------------------')
    print('Pour le train de depart : ',train)
    for c in range(1,nombre_creneaux+1):
        if form[(train,c)].x==1:
            print('For de {} se fait au créneau {} :'.format(train,c))
        if deg[(train,c)].x==1:
            print('deg de {} se fait au créneau {} :'.format(train,c))
        if oform[(train,c)].x==1:
            creneaux_occup_for.append(c)
        if odep[(train,c)].x==1:
            creneaux_occup_dep.append(c)
    print('Il occupe le chantier formation dans les horaires: ',creneaux_occup_for)
    print('Il occupe le chantier dep dans les horaires: ',creneaux_occup_dep)
    print('Ce train est lié aux trains d arrivee: ', At[train])

    print('Depart a ',creneau_depart[train])
    print('-------------------------------------------------------')